# Sanctions ME Append Demo

In [1]:
import os
import pyodbc

from ME_Append import get_me_nbr
from MarkLogicConnection import MarkLogicConnection

# Connections

### AIMS

In [2]:
auth_aims = os.environ.get('auth_aims')
aims_user, aims_pass = (auth_aims.split())
AIMS_conn = pyodbc.connect('DSN=aims_prod; UID={}; PWD={}'.format(aims_user, aims_pass))
AIMS_conn.execute('SET ISOLATION TO DIRTY READ;')

### MarkLogic

In [12]:
auth_marklogic = os.environ.get('auth_marklogic_test')
ml_user, ml_pass = (auth_marklogic.split())

url = 'http://appt1456:8000/LATEST'

MarkLogicCon = MarkLogicConnection(url=url, username=ml_user, password=ml_pass)

### Example URI

In [4]:
uri = '/json/2019.10.30_OH_NA_SL/schwartz_2172.json'

### Helper function - get State info from URI

In [5]:
def get_state(uri):   
    _sl_index = uri.index('_SL/')
    
    return uri[_sl_index-5:_sl_index-3]

In [6]:
get_state(uri)  # shown example

'OH'

# View PDF to see License Number

In [7]:
#
#
#
#
#
#

lic_nbr = '35050898'

# Meta data

In [8]:
import json

In [9]:
# clear fields for demo
MarkLogicCon.set_lic_nbr(uri=uri, lic_nbr=None)
MarkLogicCon.set_me_nbr(uri=uri, me_nbr=None)

204
204


### Pre-license-filling

In [14]:
json.loads(MarkLogicCon.get_file(uri))

{'class': 'sanction',
 'sanction': {'batch': '2019.10.30_OH_NA_SL',
  'id': '2019.10.30_OH_NA_SL/schwartz_2172',
  'date_board_order': '2019-10-31T00:00:00',
  'date_effective': None,
  'decision': None,
  'reasoning': None,
  'pas_code': None,
  'method': 'web',
  'orig': 'results/results_10_30_2019_09_00PM/OH_SummaryList_09_30PM_October_30_2019/OH-Paul_Schwartz-10_30_2019_BO.pdf',
  'physician': {'name_first': 'Paul',
   'name_middle': None,
   'name_last': 'Schwartz',
   'license': '35050898',
   'me': None}},
 'app': {'assignment': {'me': None, 'pas': None},
  'status': {'state': 'incomplete', 'me_complete': None, 'pas_complete': None},
  'meta': {'alert': None, 'note': None}}}

In [ ]:
json.loads(MarkLogicCon.get_file(uri))

### Post-license-filling

In [11]:
MarkLogicCon.set_lic_nbr(uri=uri, lic_nbr=lic_nbr)  # simulate lic_nbr update from web-app
json.loads(MarkLogicCon.get_file(uri))  # show file

204


{'class': 'sanction',
 'sanction': {'batch': '2019.10.30_OH_NA_SL',
  'id': '2019.10.30_OH_NA_SL/schwartz_2172',
  'date_board_order': '2019-10-31T00:00:00',
  'date_effective': None,
  'decision': None,
  'reasoning': None,
  'pas_code': None,
  'method': 'web',
  'orig': 'results/results_10_30_2019_09_00PM/OH_SummaryList_09_30PM_October_30_2019/OH-Paul_Schwartz-10_30_2019_BO.pdf',
  'physician': {'name_first': 'Paul',
   'name_middle': None,
   'name_last': 'Schwartz',
   'license': '35050898',
   'me': None}},
 'app': {'assignment': {'me': None, 'pas': None},
  'status': {'state': None, 'me_complete': None, 'pas_complete': None},
  'meta': {'alert': None, 'note': None}}}

### ME Lookup

In [15]:
# get metadata
json_data = json.loads(MarkLogicCon.get_file(uri))

# extract from json_data
first = json_data['sanction']['physician']['name_first']
last = json_data['sanction']['physician']['name_last']
state = get_state(uri)

print('First: {}, Last: {}, State: {}, License: {}'.format(first, last, state, lic_nbr))

First: Paul, Last: Schwartz, State: OH, License: 35050898


In [16]:
# perform ME lookup
me_nbr = get_me_nbr(first=first, last=last, state=state, lic_nbr=lic_nbr, con=AIMS_conn)

me_nbr

'03841831585'

### Fill in ME number in the meta data

In [17]:
MarkLogicCon.set_me_nbr(uri=uri, me_nbr=me_nbr)

204


### Verify / show results

In [15]:
json.loads(MarkLogicCon.get_file(uri))

{'class': 'sanction',
 'sanction': {'batch': '2019.10.30_OH_NA_SL',
  'id': '2019.10.30_OH_NA_SL/schwartz_2172',
  'date_board_order': '2019-10-31T00:00:00',
  'date_effective': None,
  'decision': None,
  'reasoning': None,
  'pas_code': None,
  'method': 'web',
  'orig': 'results/results_10_30_2019_09_00PM/OH_SummaryList_09_30PM_October_30_2019/OH-Paul_Schwartz-10_30_2019_BO.pdf',
  'physician': {'name_first': 'Paul',
   'name_middle': None,
   'name_last': 'Schwartz',
   'license': '35050898',
   'me': '03841831585'}},
 'app': {'assignment': {'me': '03841831585', 'pas': None},
  'status': {'state': None, 'me_complete': None, 'pas_complete': None},
  'meta': {'alert': None, 'note': None}}}